# Hierarchical Clustering

**Hierarchical clustering** refers to a class of clustering methods that seek to build a **hierarchy** of clusters, in which some clusters contain others. In this assignment, we will explore a top-down approach, recursively bipartitioning the data using k-means.

## Import packages

The following code block will check if you have the correct version of GraphLab Create. Any version later than 1.8.5 will do. To upgrade, read [this page](https://turi.com/download/upgrade-graphlab-create.html).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import time
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
%matplotlib inline


## Load the Wikipedia dataset

In [2]:
wiki = pd.read_csv('people_wiki.csv')

As we did in previous assignments, let's extract the TF-IDF features:

In [3]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')
map_index_to_word = pd.read_json('people_wiki_map_index_to_word.json',typ='series')
#word to index

In [30]:
map_index_to_word.index[193434]

u'fazul'

To run k-means on this dataset, we should convert the data matrix into a sparse matrix.

To be consistent with the k-means assignment, let's normalize all vectors to have unit norm.

In [4]:
from sklearn.preprocessing import normalize
tf_idf = normalize(tf_idf)

## Bipartition the Wikipedia dataset using k-means

Recall our workflow for clustering text data with k-means:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with some value of k.
4. Visualize the clustering results using the centroids, cluster assignments, and the original dataframe. We keep the original dataframe around because the data matrix does not keep auxiliary information (in the case of the text dataset, the title of each article).

Let us modify the workflow to perform bipartitioning:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with k=2.
4. Divide the data matrix into two parts using the cluster assignments.
5. Divide the dataframe into two parts, again using the cluster assignments. This step is necessary to allow for visualization.
6. Visualize the bipartition of data.

We'd like to be able to repeat Steps 3-6 multiple times to produce a **hierarchy** of clusters such as the following:
```
                      (root)
                         |
            +------------+-------------+
            |                          |
         Cluster                    Cluster
     +------+-----+             +------+-----+
     |            |             |            |
   Cluster     Cluster       Cluster      Cluster
```
Each **parent cluster** is bipartitioned to produce two **child clusters**. At the very top is the **root cluster**, which consists of the entire dataset.

Now we write a wrapper function to bipartition a given cluster using k-means. There are three variables that together comprise the cluster:

* `dataframe`: a subset of the original dataframe that correspond to member rows of the cluster
* `matrix`: same set of rows, stored in sparse matrix format
* `centroid`: the centroid of the cluster (not applicable for the root cluster)

Rather than passing around the three variables separately, we package them into a Python dictionary. The wrapper function takes a single dictionary (representing a parent cluster) and returns two dictionaries (representing the child clusters).

In [7]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be a dictionary containing the following keys
                * dataframe: original dataframe
                * matrix:    same data, in matrix format
                * centroid:  centroid for this particular cluster'''
    
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    
    # Run k-means on the data matrix with k=2. We use scikit-learn here to simplify workflow.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=-1, verbose=1)
    kmeans_model.fit(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], \
                                                      data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments.
    cluster_assignment_sa = np.array(cluster_assignment) # minor format conversion
    dataframe_left_child, dataframe_right_child     = dataframe[cluster_assignment_sa==0], \
                                                      dataframe[cluster_assignment_sa==1]
        
    
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}
    
    return (cluster_left_child, cluster_right_child)

The following cell performs bipartitioning of the Wikipedia dataset. Allow 20-60 seconds to finish.

Note. For the purpose of the assignment, we set an explicit seed (`seed=1`) to produce identical outputs for every run. In pratical applications, you might want to use different random seeds for all runs.

In [8]:
wiki_data = {'matrix': tf_idf, 'dataframe': wiki} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=6, seed=1)

Initialization complete
Initialization complete
Initialization complete
Initialization complete
Iteration  0, inertia 116092.674
Iteration  0, inertia 116351.246
Iteration  0, inertia 116293.919
Iteration  0, inertia 116101.162
Iteration  1, inertia 58254.170
Iteration  1, inertia 58294.624
Iteration  1, inertia 58271.393
Iteration  1, inertia 58247.691
Iteration  2, inertia 58230.098
Iteration  2, inertia 58233.830
Iteration  2, inertia 58242.910
Iteration  2, inertia 58228.673
Iteration  3, inertia 58226.274
Iteration  3, inertia 58227.713
Iteration  3, inertia 58232.422
Iteration  3, inertia 58225.418
Iteration  4, inertia 58224.964
Iteration  4, inertia 58226.204
Iteration  4, inertia 58228.131
Iteration  4, inertia 58224.401
Iteration  5, inertia 58224.668
Iteration  5, inertia 58225.580
Iteration  5, inertia 58226.902
Iteration  5, inertia 58223.619
Iteration  6, inertia 58224.616
Iteration  6, inertia 58225.205
Iteration  6, inertia 58222.908
Iteration  6, inertia 58226.284
Iter

Let's examine the contents of one of the two clusters, which we call the `left_child`, referring to the tree visualization above.

In [9]:
left_child

{'centroid': array([  0.00000000e+00,   8.57526623e-06,   0.00000000e+00, ...,
          1.38560691e-04,   6.46049863e-05,   2.26551103e-05]),
 'dataframe':                                                      URI  \
 0            <http://dbpedia.org/resource/Digby_Morrell>   
 17     <http://dbpedia.org/resource/Paddy_Dunne_(Gael...   
 21           <http://dbpedia.org/resource/Ceiron_Thomas>   
 22            <http://dbpedia.org/resource/Adel_Sellimi>   
 25             <http://dbpedia.org/resource/Vic_Stasiuk>   
 28            <http://dbpedia.org/resource/Leon_Hapgood>   
 30               <http://dbpedia.org/resource/Dom_Flora>   
 33               <http://dbpedia.org/resource/Bob_Reece>   
 41     <http://dbpedia.org/resource/Bob_Adams_(Americ...   
 48              <http://dbpedia.org/resource/Marc_Logan>   
 49          <http://dbpedia.org/resource/Corey_Woolfolk>   
 63              <http://dbpedia.org/resource/Alan_Roper>   
 75      <http://dbpedia.org/resource/Vladimir_Yurc

And here is the content of the other cluster we named `right_child`.

In [10]:
right_child

{'centroid': array([  3.00882137e-06,   0.00000000e+00,   2.88868244e-06, ...,
          1.10291526e-04,   9.00609890e-05,   2.03703564e-05]),
 'dataframe':                                                      URI  \
 1           <http://dbpedia.org/resource/Alfred_J._Lewy>   
 2            <http://dbpedia.org/resource/Harpdog_Brown>   
 3      <http://dbpedia.org/resource/Franz_Rottensteiner>   
 4                   <http://dbpedia.org/resource/G-Enka>   
 5            <http://dbpedia.org/resource/Sam_Henderson>   
 6            <http://dbpedia.org/resource/Aaron_LaCrate>   
 7          <http://dbpedia.org/resource/Trevor_Ferguson>   
 8             <http://dbpedia.org/resource/Grant_Nelson>   
 9             <http://dbpedia.org/resource/Cathy_Caruth>   
 10            <http://dbpedia.org/resource/Sophie_Crumb>   
 11           <http://dbpedia.org/resource/Jenn_Ashworth>   
 12        <http://dbpedia.org/resource/Jonathan_Hoefler>   
 13     <http://dbpedia.org/resource/Anthony_Gueter

## Visualize the bipartition

We provide you with a modified version of the visualization function from the k-means assignment. For each cluster, we print the top 5 words with highest TF-IDF weights in the centroid and display excerpts for the 8 nearest neighbors of the centroid.

In [35]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: SFrame specifying the mapping betweeen words and column indices'''
    
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster
    idx = centroid.argsort()[::-1]
    for i in xrange(5):
        print('{0:s}:{1:.3f}'.format(map_index_to_word.index[idx[i]], centroid[idx[i]])),
    print('')
    
    # Compute distances from the centroid to all data points in the cluster.
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    # compute nearest neighbors of the centroid within the cluster.
    nearest_neighbors = distances.argsort()
    # For 8 nearest neighbors, print the title as well as first 180 characters of text.
    # Wrap the text at 80-character mark.
    for i in xrange(8):
        text = ' '.join(wiki_subset.iloc[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset.iloc[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')

In [36]:
left_child['centroid']

array([  0.00000000e+00,   8.57526623e-06,   0.00000000e+00, ...,
         1.38560691e-04,   6.46049863e-05,   2.26551103e-05])

In [41]:
left_child['centroid'].shape

(547979,)

In [42]:
tf_idf.shape

(59071, 547979)

In [43]:
sum(left_child['centroid'])

8.6960888147290429

Let's visualize the two child clusters:

In [44]:
display_single_tf_idf_cluster(left_child, map_index_to_word)

zvuku:0.040 zwerge:0.036 zwines:0.029 zumars:0.029 zx10rborn:0.028 
* Todd Williams                                      0.95468
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Gord Sherven                                       0.95622
  gordon r sherven born august 21 1963 in gravelbourg saskatchewan and raised in mankota sas
  katchewan is a retired canadian professional ice hockey forward who played
* Justin Knoedler                                    0.95639
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Chris Day                                          0.95648
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Tony Smith (footba

In [45]:
display_single_tf_idf_cluster(right_child, map_index_to_word)

zwolsman:0.025 zx10r:0.017 zwigoff:0.012 zyuganovs:0.011 zyntherius:0.011 
* Anita Kunz                                         0.97401
  anita e kunz oc born 1956 is a canadianborn artist and illustratorkunz has lived in london
   new york and toronto contributing to magazines and working
* Janet Jackson                                      0.97472
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Madonna (entertainer)                              0.97475
  madonna louise ciccone tkoni born august 16 1958 is an american singer songwriter actress 
  and businesswoman she achieved popularity by pushing the boundaries of lyrical
* %C3%81ine Hyland                                   0.97536
  ine hyland ne donlon is emeritus professor of education and former vicepresident of univer
  sity college cork ireland she was born in 1942 in athboy co
* Jane Fonda                            

The left cluster consists of athletes, whereas the right cluster consists of non-athletes. So far, we have a single-level hierarchy consisting of two clusters, as follows:

```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
                 Athletes                                      Non-athletes
```

Is this hierarchy good enough? **When building a hierarchy of clusters, we must keep our particular application in mind.** For instance, we might want to build a **directory** for Wikipedia articles. A good directory would let you quickly narrow down your search to a small set of related articles. The categories of athletes and non-athletes are too general to facilitate efficient search. For this reason, we decide to build another level into our hierarchy of clusters with the goal of getting more specific cluster structure at the lower level. To that end, we subdivide both the `athletes` and `non-athletes` clusters.

## Perform recursive bipartitioning

### Cluster of athletes

To help identify the clusters we've built so far, let's give them easy-to-read aliases:

In [46]:
athletes = left_child
non_athletes = right_child

Using the bipartition function, we produce two child clusters of the athlete cluster:

In [47]:
# Bipartition the cluster of athletes
left_child_athletes, right_child_athletes = bipartition(athletes, maxiter=100, num_runs=6, seed=1)

Initialization complete
Initialization complete
Initialization complete
Initialization complete
Iteration  0, inertia 22100.232
Iteration  0, inertia 22241.279
Iteration  0, inertia 21717.211
Iteration  0, inertia 21964.990
Iteration  1, inertia 11163.410
Iteration  1, inertia 11144.817
Iteration  1, inertia 11119.067
Iteration  1, inertia 11160.597
Iteration  2, inertia 11134.998
Iteration  2, inertia 11144.691
Iteration  2, inertia 11105.094
Iteration  2, inertia 11143.919
Iteration  3, inertia 11125.568
Iteration  3, inertia 11134.000
Iteration  3, inertia 11103.271
Iteration  3, inertia 11138.340
Iteration  4, inertia 11120.867
Iteration  4, inertia 11133.814
Iteration  4, inertia 11103.108
Iteration  4, inertia 11135.742
Iteration  5, inertia 11120.533
Iteration  5, inertia 11133.765
Iteration  5, inertia 11103.103
Converged at iteration 5: center shift 0.000000e+00 within tolerance 1.773880e-10
Iteration  5, inertia 11135.108
Initialization complete
Iteration  6, inertia 11133.75

The left child cluster mainly consists of baseball players:

In [48]:
display_single_tf_idf_cluster(left_child_athletes, map_index_to_word)

zvuku:0.054 zwerge:0.043 zumars:0.038 zx10rborn:0.035 zuidams:0.030 
* Tony Smith (footballer, born 1957)                 0.94677
  anthony tony smith born 20 february 1957 is a former footballer who played as a central de
  fender in the football league in the 1970s and
* Justin Knoedler                                    0.94746
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Chris Day                                          0.94849
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Todd Williams                                      0.94882
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Todd Curley                                  

On the other hand, the right child cluster is a mix of players in association football, Austrailian rules football and ice hockey:

In [49]:
display_single_tf_idf_cluster(right_child_athletes, map_index_to_word)

zowie:0.045 zuberi:0.043 zululand:0.035 zyiit:0.031 zygouli:0.031 
* Alessandra Aguilar                                 0.93880
  alessandra aguilar born 1 july 1978 in lugo is a spanish longdistance runner who specialis
  es in marathon running she represented her country in the event
* Heather Samuel                                     0.93999
  heather barbara samuel born 6 july 1970 is a retired sprinter from antigua and barbuda who
   specialized in the 100 and 200 metres in 1990
* Viola Kibiwot                                      0.94037
  viola jelagat kibiwot born december 22 1983 in keiyo district is a runner from kenya who s
  pecialises in the 1500 metres kibiwot won her first
* Ayelech Worku                                      0.94052
  ayelech worku born june 12 1979 is an ethiopian longdistance runner most known for winning
   two world championships bronze medals on the 5000 metres she
* Krisztina Papp                                     0.94105
  krisztina papp born 1

Our hierarchy of clusters now looks like this:
```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
                 Athletes                                      Non-athletes
                    +
                    |
        +-----------+--------+
        |                    |
        |            association football/
        +          Austrailian rules football/
     baseball             ice hockey
```

Should we keep subdividing the clusters? If so, which cluster should we subdivide? To answer this question, we again think about our application. Since we organize our directory by topics, it would be nice to have topics that are about as coarse as each other. For instance, if one cluster is about baseball, we expect some other clusters about football, basketball, volleyball, and so forth. That is, **we would like to achieve similar level of granularity for all clusters.**

Notice that the right child cluster is more coarse than the left child cluster. The right cluster posseses a greater variety of topics than the left (ice hockey/association football/Austrialian football vs. baseball). So the right child cluster should be subdivided further to produce finer child clusters.

Let's give the clusters aliases as well:

In [50]:
baseball            = left_child_athletes
ice_hockey_football = right_child_athletes

### Cluster of ice hockey players and football players

In answering the following quiz question, take a look at the topics represented in the top documents (those closest to the centroid), as well as the list of words with highest TF-IDF weights.

Let us bipartition the cluster of ice hockey and football players.

In [51]:
left_child_ihs, right_child_ihs = bipartition(ice_hockey_football, maxiter=100, num_runs=6, seed=1)

Initialization complete
Initialization complete
Initialization complete
Initialization complete
Iteration  0, inertia 5545.820
Iteration  0, inertia 5669.348
Iteration  0, inertia 5690.017
Iteration  0, inertia 5677.596
Iteration  1, inertia 2864.434
Iteration  1, inertia 2840.933
Iteration  1, inertia 2857.828
Iteration  1, inertia 2868.816
Iteration  2, inertia 2842.140
Iteration  2, inertia 2848.948
Iteration  2, inertia 2837.362
Iteration  2, inertia 2855.981
Iteration  3, inertia 2839.179
Iteration  3, inertia 2841.765
Iteration  3, inertia 2852.776
Iteration  3, inertia 2837.315
Iteration  4, inertia 2838.790
Iteration  4, inertia 2837.081
Iteration  4, inertia 2852.441
Iteration  4, inertia 2837.303
Iteration  5, inertia 2837.344
Iteration  5, inertia 2852.120
Iteration  5, inertia 2837.054
Iteration  5, inertia 2837.300
Converged at iteration 5: center shift 0.000000e+00 within tolerance 1.754910e-10
Initialization complete
Iteration  6, inertia 2837.324
Iteration  6, inertia 2

In [52]:
display_single_tf_idf_cluster(left_child_ihs, map_index_to_word)

zowie:0.064 zyiit:0.039 zwolsman:0.038 zealandamerican:0.038 zolecki:0.037 
* Heather Samuel                                     0.91590
  heather barbara samuel born 6 july 1970 is a retired sprinter from antigua and barbuda who
   specialized in the 100 and 200 metres in 1990
* Krisztina Papp                                     0.91672
  krisztina papp born 17 december 1982 in eger is a hungarian long distance runner she is th
  e national indoor record holder over 5000 mpapp began
* Ayelech Worku                                      0.91892
  ayelech worku born june 12 1979 is an ethiopian longdistance runner most known for winning
   two world championships bronze medals on the 5000 metres she
* Viola Kibiwot                                      0.91906
  viola jelagat kibiwot born december 22 1983 in keiyo district is a runner from kenya who s
  pecialises in the 1500 metres kibiwot won her first
* Alessandra Aguilar                                 0.91955
  alessandra aguilar bor

In [53]:
display_single_tf_idf_cluster(right_child_ihs, map_index_to_word)

zuberi:0.118 zadnji:0.089 zimmermann:0.062 zekiye:0.060 zululand:0.054 
* Bob Heintz                                         0.88057
  robert edward heintz born may 1 1970 is an american professional golfer who plays on the n
  ationwide tourheintz was born in syosset new york he
* Tim Conley                                         0.88274
  tim conley born december 8 1958 is an american professional golfer who played on the pga t
  our nationwide tour and most recently the champions tourconley
* Bruce Zabriski                                     0.88279
  bruce zabriski born august 3 1957 is an american professional golfer who played on the pga
   tour european tour and the nationwide tourzabriski joined the
* Sonny Skinner                                      0.88438
  sonny skinner born august 18 1960 is an american professional golfer who plays on the cham
  pions tourskinner was born in portsmouth virginia he turned professional
* Brian Davis (golfer)                              

**Quiz Question**. Which diagram best describes the hierarchy right after splitting the `ice_hockey_football` cluster? Refer to the quiz form for the diagrams.

**Caution**. The granularity criteria is an imperfect heuristic and must be taken with a grain of salt. It takes a lot of manual intervention to obtain a good hierarchy of clusters.

* **If a cluster is highly mixed, the top articles and words may not convey the full picture of the cluster.** Thus, we may be misled if we judge the purity of clusters solely by their top documents and words. 
* **Many interesting topics are hidden somewhere inside the clusters but do not appear in the visualization.** We may need to subdivide further to discover new topics. For instance, subdividing the `ice_hockey_football` cluster led to the appearance of runners and golfers.

### Cluster of non-athletes

Now let us subdivide the cluster of non-athletes.

In [54]:
# Bipartition the cluster of non-athletes
left_child_non_athletes, right_child_non_athletes = bipartition(non_athletes, maxiter=100, num_runs=6, seed=1)

Initialization complete
Initialization complete
Initialization complete
Initialization complete
Iteration  0, inertia 93346.712
Iteration  0, inertia 93161.798
Iteration  0, inertia 93170.164
Iteration  0, inertia 93261.291
Iteration  1, inertia 46885.838
Iteration  1, inertia 46884.698
Iteration  1, inertia 46929.611
Iteration  1, inertia 46896.417
Iteration  2, inertia 46848.998
Iteration  2, inertia 46847.112
Iteration  2, inertia 46864.557
Iteration  2, inertia 46852.047
Iteration  3, inertia 46846.011
Iteration  3, inertia 46845.375
Iteration  3, inertia 46856.212
Iteration  3, inertia 46846.735
Iteration  4, inertia 46845.130
Iteration  4, inertia 46845.156
Iteration  4, inertia 46853.087
Iteration  4, inertia 46845.435
Iteration  5, inertia 46844.878
Iteration  5, inertia 46845.081
Iteration  5, inertia 46851.404
Iteration  5, inertia 46845.126
Iteration  6, inertia 46844.801
Iteration  6, inertia 46845.038
Iteration  6, inertia 46850.774
Iteration  6, inertia 46845.041
Iteratio

In [55]:
display_single_tf_idf_cluster(left_child_non_athletes, map_index_to_word)

zyntherius:0.016 zyuganovs:0.013 zwolsman:0.013 zupan:0.012 zx81:0.012 
* Barry Sullivan (lawyer)                            0.97227
  barry sullivan is a chicago lawyer and as of july 1 2009 the cooney conway chair in advoca
  cy at loyola university chicago school of law
* Kayee Griffin                                      0.97444
  kayee frances griffin born 6 february 1950 is an australian politician and former australi
  an labor party member of the new south wales legislative council serving
* Christine Robertson                                0.97450
  christine mary robertson born 5 october 1948 is an australian politician and former austra
  lian labor party member of the new south wales legislative council serving
* James A. Joseph                                    0.97464
  james a joseph born 1935 is an american former diplomatjoseph is professor of the practice
   of public policy studies at duke university and founder of
* David Anderson (British Columbia politician)    

In [56]:
display_single_tf_idf_cluster(right_child_non_athletes, map_index_to_word)

zwolsman:0.039 zx10r:0.030 zwigoff:0.023 zwacksalles:0.021 zupanprofessor:0.015 
* Madonna (entertainer)                              0.96092
  madonna louise ciccone tkoni born august 16 1958 is an american singer songwriter actress 
  and businesswoman she achieved popularity by pushing the boundaries of lyrical
* Janet Jackson                                      0.96153
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Cher                                               0.96540
  cher r born cherilyn sarkisian may 20 1946 is an american singer actress and television ho
  st described as embodying female autonomy in a maledominated industry
* Laura Smith                                        0.96600
  laura smith is a canadian folk singersongwriter she is best known for her 1995 single shad
  e of your love one of the years biggest hits
* Natashia Williams                    

Neither of the clusters show clear topics, apart from the genders. Let us divide them further.

In [57]:
male_non_athletes = left_child_non_athletes
female_non_athletes = right_child_non_athletes

In [58]:
# Bipartition the cluster of non-athletes
left_child_male_non_athletes, right_child_male_non_athletes = bipartition(male_non_athletes, maxiter=100, num_runs=6, seed=1)

Initialization complete
Initialization complete
Initialization complete
Initialization complete
Iteration  0, inertia 48531.918
Iteration  0, inertia 48311.627
Iteration  0, inertia 48273.633
Iteration  0, inertia 48365.413
Iteration  1, inertia 24317.797
Iteration  1, inertia 24319.179
Iteration  1, inertia 24334.175
Iteration  1, inertia 24306.421
Iteration  2, inertia 24291.546
Iteration  2, inertia 24310.127
Iteration  2, inertia 24296.701
Iteration  2, inertia 24298.458
Iteration  3, inertia 24301.189
Iteration  3, inertia 24286.515
Iteration  3, inertia 24296.419
Iteration  3, inertia 24295.281
Iteration  4, inertia 24284.880
Iteration  4, inertia 24297.907
Iteration  4, inertia 24295.556
Iteration  4, inertia 24294.428
Iteration  5, inertia 24284.052
Iteration  5, inertia 24294.829
Iteration  5, inertia 24293.195
Iteration  5, inertia 24295.978
Iteration  6, inertia 24293.846
Iteration  6, inertia 24294.186
Iteration  6, inertia 24291.134
Iteration  6, inertia 24283.474
Iteratio

In [59]:
display_single_tf_idf_cluster(left_child_male_non_athletes, map_index_to_word)

zurichminds:0.039 zumadespite:0.036 zuffelato:0.033 zwolsman:0.028 zuzana:0.026 
* Kayee Griffin                                      0.95170
  kayee frances griffin born 6 february 1950 is an australian politician and former australi
  an labor party member of the new south wales legislative council serving
* Marcelle Mersereau                                 0.95417
  marcelle mersereau born february 14 1942 in pointeverte new brunswick is a canadian politi
  cian a civil servant for most of her career she also served
* Lucienne Robillard                                 0.95453
  lucienne robillard pc born june 16 1945 is a canadian politician and a member of the liber
  al party of canada she sat in the house
* Maureen Lyster                                     0.95590
  maureen anne lyster born 10 september 1943 is an australian politician she was an australi
  an labor party member of the victorian legislative assembly from 1985
* Liz Cunningham                                    

In [60]:
display_single_tf_idf_cluster(right_child_male_non_athletes, map_index_to_word)

zyntherius:0.018 zvezdy:0.015 zwane:0.013 zyuganovs:0.013 zwicky:0.010 
* Lawrence W. Green                                  0.97495
  lawrence w green is best known by health education researchers as the originator of the pr
  ecede model and codeveloper of the precedeproceed model which has
* James A. Joseph                                    0.97506
  james a joseph born 1935 is an american former diplomatjoseph is professor of the practice
   of public policy studies at duke university and founder of
* Timothy Luke                                       0.97584
  timothy w luke is university distinguished professor of political science in the college o
  f liberal arts and human sciences as well as program chair of
* Archie Brown                                       0.97628
  archibald haworth brown cmg fba commonly known as archie brown born 10 may 1938 is a briti
  sh political scientist and historian in 2005 he became
* Jerry L. Martin                                    0.97687


In [61]:
# Bipartition the cluster of non-athletes
left_child_female_non_athletes, right_child_female_non_athletes = bipartition(female_non_athletes, maxiter=100, num_runs=6, seed=1)

Initialization complete
Initialization complete
Initialization complete
Initialization complete
Iteration  0, inertia 44853.060
Iteration  0, inertia 44679.078
Iteration  0, inertia 44839.016
Iteration  0, inertia 44467.930
Iteration  1, inertia 22445.702
Iteration  1, inertia 22393.793
Iteration  1, inertia 22437.377
Iteration  1, inertia 22429.320
Iteration  2, inertia 22407.887
Iteration  2, inertia 22416.328
Iteration  2, inertia 22366.020
Iteration  2, inertia 22418.303
Iteration  3, inertia 22393.069
Iteration  3, inertia 22404.650
Iteration  3, inertia 22358.156
Iteration  3, inertia 22414.452
Iteration  4, inertia 22380.942
Iteration  4, inertia 22402.987
Iteration  4, inertia 22357.405
Iteration  4, inertia 22409.087
Iteration  5, inertia 22372.280
Iteration  5, inertia 22402.806
Iteration  5, inertia 22357.315
Iteration  5, inertia 22403.931
Iteration  6, inertia 22363.775
Iteration  6, inertia 22357.305
Iteration  6, inertia 22402.770
Iteration  6, inertia 22400.398
Iteratio

In [62]:
display_single_tf_idf_cluster(left_child_female_non_athletes, map_index_to_word)

zwigoff:0.027 zwacksalles:0.023 zupanprofessor:0.017 zulfiqar:0.016 zuppiger:0.015 
* Julian Knowles                                     0.96904
  julian knowles is an australian composer and performer specialising in new and emerging te
  chnologies his creative work spans the fields of composition for theatre dance
* Peter Combe                                        0.97080
  peter combe born 20 october 1948 is an australian childrens entertainer and musicianmusica
  l genre childrens musiche has had 22 releases including seven gold albums two
* Craig Pruess                                       0.97121
  craig pruess born 1950 is an american composer musician arranger and gold platinum record 
  producer who has been living in britain since 1973 his career
* Ceiri Torjussen                                    0.97169
  ceiri torjussen born 1976 is a composer who has contributed music to dozens of film and te
  levision productions in the ushis music was described by
* Brenton Broads

In [63]:
display_single_tf_idf_cluster(right_child_female_non_athletes, map_index_to_word)

zwolsman:0.124 zx10r:0.092 zwacksalles:0.015 zongkar:0.015 zwigoff:0.014 
* Janet Jackson                                      0.93374
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Barbara Hershey                                    0.93507
  barbara hershey born barbara lynn herzstein february 5 1948 once known as barbara seagull 
  is an american actress in a career spanning nearly 50 years
* Lauren Royal                                       0.93717
  lauren royal born march 3 circa 1965 is a book writer from california royal has written bo
  th historic and novelistic booksa selfproclaimed angels baseball fan
* Alexandra Potter                                   0.93802
  alexandra potter born 1970 is a british author of romantic comediesborn in bradford yorksh
  ire england and educated at liverpool university gaining an honors degree in
* Cher                            

**Quiz Question**. Let us bipartition the clusters `male_non_athletes` and `female_non_athletes`. Which diagram best describes the resulting hierarchy of clusters for the non-athletes? Refer to the quiz for the diagrams.

**Note**. Use `maxiter=100, num_runs=6, seed=1` for consistency of output.